In [188]:
import warnings
warnings.filterwarnings("ignore")

In [121]:
from gc import enable

import pandas as pd
from google.cloud import bigquery
from google.cloud import bigquery_storage
creds = "/Users/srdeo/OneDrive - Copart, Inc/secrets/cprtpr-datastewards-sp1-614d7e297848 (1).json"
client = bigquery.Client.from_service_account_json(json_credentials_path=creds)
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=creds)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [122]:

query1 = """WITH non_churn AS (
    SELECT
        m.owner_mbr_nbr,
        m.mbr_nbr,
        m.mbr_type,
        m.mbr_mbrshp_type_cd,
        e.event_type,
        e.event_value,
        0 AS churn
    FROM `cprtpr-dataplatform-sp1`.usviews.v_us_member_fact m
    JOIN `cprtpr-dataplatform-sp1`.usmart.v_us_member_events_fact e
        ON m.mbr_nbr = e.mbr_nbr
    WHERE m.mbr_type = 'PREMIER'
      AND e.event_value BETWEEN '2025-04-01' AND '2025-04-30'
      AND e.event_type = 'Member Premier Renewal Paid Date'
),
churn AS (
    SELECT
        m.owner_mbr_nbr,
        m.mbr_nbr,
        m.mbr_type,
        m.mbr_mbrshp_type_cd,
        e.event_type,
        e.event_value,
        1 AS churn
    FROM `cprtpr-dataplatform-sp1`.usviews.v_us_member_fact m
    JOIN `cprtpr-dataplatform-sp1`.usmart.v_us_member_events_fact e
        ON m.mbr_nbr = e.mbr_nbr
    WHERE m.mbr_type = 'LPSDPREMIER'
      AND e.event_value BETWEEN '2025-04-01' AND '2025-04-30'
      AND e.event_type = 'MEMBER TYPE UPDATED FROM PREMIER TO GUEST'
)

SELECT * FROM non_churn
UNION ALL
SELECT * FROM churn;
"""

data1 = client.query(query1).to_dataframe()

In [123]:
data1
print(data1.shape)
print(data1.isnull().sum())
print(data1['churn'].value_counts())

,owner_mbr_nbr,mbr_nbr,mbr_type,mbr_mbrshp_type_cd,event_type,event_value,churn
0,696204,696204,LPSDPREMIER,GUEST,MEMBER TYPE UPDATED FROM PREMIER TO GUEST,2025-04-20 23:51:09+00:00,1
1,175159,175159,LPSDPREMIER,GUEST,MEMBER TYPE UPDATED FROM PREMIER TO GUEST,2025-04-04 10:00:31+00:00,1
2,523173,523173,LPSDPREMIER,GUEST,MEMBER TYPE UPDATED FROM PREMIER TO GUEST,2025-04-28 10:00:30+00:00,1
3,551689,551689,LPSDPREMIER,GUEST,MEMBER TYPE UPDATED FROM PREMIER TO GUEST,2025-04-04 10:00:48+00:00,1
4,390132,390132,LPSDPREMIER,GUEST,MEMBER TYPE UPDATED FROM PREMIER TO GUEST,2025-04-03 10:00:45+00:00,1
...,...,...,...,...,...,...,...
6430,99426,99426,PREMIER,PREMIER,Member Premier Renewal Paid Date,2025-04-12 18:09:30+00:00,0
6431,40736,40736,PREMIER,PREMIER,Member Premier Renewal Paid Date,2025-04-09 20:48:20+00:00,0
6432,503146,503146,PREMIER,PREMIER,Member Premier Renewal Paid Date,2025-04-02 00:31:48+00:00,0
6433,685482,685482,PREMIER,PREMIER,Member Premier Renewal Paid Date,2025-04-09 16:28:49+00:00,0


(6435, 7)
owner_mbr_nbr         0
mbr_nbr               0
mbr_type              0
mbr_mbrshp_type_cd    0
event_type            0
event_value           0
churn                 0
dtype: int64
churn
1    4002
0    2433
Name: count, dtype: Int64


In [124]:
churn_members = data1[data1['churn'] == 1]['owner_mbr_nbr'].unique().tolist()
not_churn_members = data1[data1['churn'] == 0]['owner_mbr_nbr'].unique().tolist()

In [126]:
len(churn_members), len(not_churn_members)

(3984, 2433)

In [189]:
# Combine both lists and remove duplicates
all_members = pd.unique(list(churn_members) + list(not_churn_members)).tolist()

* Churned members Analysis:
  * How many total bids placed on lots sold between Jan-March?
  * How many lots did they bid on between Jan-March?
  * How many lots did they win between Jan-March?

In [128]:
all_members_list = [str(x) for x in all_members]  # convert to string
all_members_str = ", ".join(all_members_list)     # no quotes around ints

In [129]:
# Final query
query2 = f"""
WITH bids_placed AS (
SELECT
  owner_mbr_nbr,
  m.mbr_state,
  m.member_sgmnt_type,
  COUNT(DISTINCT b.bid_id) AS total_bids_on_lots_sold,
  COUNT(DISTINCT b.lot_nbr) AS total_lots_placed_bids_on
FROM `cprtpr-dataplatform-sp1`.usviews.v_us_member_fact m
LEFT JOIN `cprtpr-dataplatform-sp1`.usviews.v_us_bids_fact b
  ON m.mbr_nbr = b.buyer_nbr
LEFT JOIN `cprtpr-dataplatform-sp1`.usviews.v_us_lot_fact l
  ON l.lot_nbr = b.lot_nbr AND l.inv_dt BETWEEN '2025-01-01' AND '2025-03-31'
WHERE owner_mbr_nbr IN ({all_members_str})
GROUP BY owner_mbr_nbr, m.mbr_state, m.member_sgmnt_type
),

lots_won AS (
  SELECT
    owner_mbr_nbr,
    COUNT(DISTINCT l.lot_nbr) AS total_lots_won
  FROM `cprtpr-dataplatform-sp1`.usviews.v_us_member_fact m
  JOIN `cprtpr-dataplatform-sp1`.usviews.v_us_lot_fact l
    ON l.buyer_nbr = m.mbr_nbr
  WHERE inv_dt BETWEEN '2025-01-01' AND '2025-03-31'
    AND owner_mbr_nbr IN ({all_members_str})
  GROUP BY owner_mbr_nbr
)

SELECT
  b.owner_mbr_nbr,
  b.total_bids_on_lots_sold,
  b.total_lots_placed_bids_on,
  w.total_lots_won,
    b.mbr_state,
    b.member_sgmnt_type
FROM bids_placed b
LEFT JOIN lots_won w
  ON b.owner_mbr_nbr = w.owner_mbr_nbr;

"""

# Run the query
data = client.query(query2).to_dataframe()

In [130]:
data
data.shape

,owner_mbr_nbr,total_bids_on_lots_sold,total_lots_placed_bids_on,total_lots_won,mbr_state,member_sgmnt_type
0,463585,1041,212,3,MN,Consumer
1,748790,102,19,<NA>,AR,Consumer
2,741214,345,25,1,MT,General Business
3,45910,37,10,<NA>,TN,General Business
4,314004,34,4,<NA>,PA,Consumer
...,...,...,...,...,...,...
6412,266896,28,5,<NA>,MN,Consumer
6413,241455,48,12,<NA>,PA,General Business
6414,513564,0,0,<NA>,OH,None
6415,223900,111,25,<NA>,AB,Exporter


(6417, 6)

In [131]:
data[data['owner_mbr_nbr']== 457]  # check one case

,owner_mbr_nbr,total_bids_on_lots_sold,total_lots_placed_bids_on,total_lots_won,mbr_state,member_sgmnt_type
4583,457,71539,9661,66,TX,Dismantler


In [132]:
## Adding churn column to the data

data['churn'] = data['owner_mbr_nbr'].isin(churn_members).astype(int)

In [133]:
data

,owner_mbr_nbr,total_bids_on_lots_sold,total_lots_placed_bids_on,total_lots_won,mbr_state,member_sgmnt_type,churn
0,463585,1041,212,3,MN,Consumer,1
1,748790,102,19,<NA>,AR,Consumer,1
2,741214,345,25,1,MT,General Business,0
3,45910,37,10,<NA>,TN,General Business,0
4,314004,34,4,<NA>,PA,Consumer,1
...,...,...,...,...,...,...,...
6412,266896,28,5,<NA>,MN,Consumer,1
6413,241455,48,12,<NA>,PA,General Business,1
6414,513564,0,0,<NA>,OH,None,1
6415,223900,111,25,<NA>,AB,Exporter,1


In [134]:
## Count of renewed vs not renewed members

data['churn'].value_counts()

churn
1    3984
0    2433
Name: count, dtype: int64

### Only shortlisting Churned Members

In [136]:
data_churn = data[data['churn'] == 1]

* List of churned members who placed at least one bid on lots sold between Jan-March

In [190]:
data_churn['bid_atleast_once'] = np.where(data_churn['total_lots_placed_bids_on'] > 0, 1, 0)

In [138]:
data_churn

,owner_mbr_nbr,total_bids_on_lots_sold,total_lots_placed_bids_on,total_lots_won,mbr_state,member_sgmnt_type,churn,bid_atleast_once
0,463585,1041,212,3,MN,Consumer,1,1
1,748790,102,19,<NA>,AR,Consumer,1,1
4,314004,34,4,<NA>,PA,Consumer,1,1
6,723585,0,0,<NA>,PA,Consumer,1,0
7,299919,0,0,<NA>,LA,Consumer,1,0
...,...,...,...,...,...,...,...,...
6412,266896,28,5,<NA>,MN,Consumer,1,1
6413,241455,48,12,<NA>,PA,General Business,1,1
6414,513564,0,0,<NA>,OH,None,1,0
6415,223900,111,25,<NA>,AB,Exporter,1,1


In [139]:
data_churn['bid_atleast_once'].value_counts()

bid_atleast_once
1    3202
0     782
Name: count, dtype: int64

* List of churned members who won at least one lot between Jan-March

In [191]:
data_churn['won_atleast_once'] = np.where(pd.isna(data_churn['total_lots_won']), 0, 1)

In [149]:
data_churn[data_churn['bid_atleast_once']==1]['won_atleast_once'].value_counts()

won_atleast_once
0    2909
1     293
Name: count, dtype: int64

### Only shortlisting non-churned members

In [161]:
data_not_churn = data[data['churn'] == 0]

In [162]:
data_not_churn
data_not_churn.shape

,owner_mbr_nbr,total_bids_on_lots_sold,total_lots_placed_bids_on,total_lots_won,mbr_state,member_sgmnt_type,churn
2,741214,345,25,1,MT,General Business,0
3,45910,37,10,<NA>,TN,General Business,0
5,813638,121,28,<NA>,OR,Consumer,0
8,535380,2878,358,18,NV,Dealer,0
10,385679,7514,1197,25,FL,Dealer,0
...,...,...,...,...,...,...,...
6401,305893,7313,1297,5,FL,Dealer,0
6402,241014,907,105,<NA>,AZ,Consumer,0
6406,573155,1997,257,1,CA,Dealer,0
6409,247892,532,108,<NA>,CA,General Business,0


(2433, 7)

In [163]:
data_not_churn[data_not_churn['owner_mbr_nbr']==158785]   ## checking one case

,owner_mbr_nbr,total_bids_on_lots_sold,total_lots_placed_bids_on,total_lots_won,mbr_state,member_sgmnt_type,churn
1546,158785,1949,291,8,WA,Consumer,0


* List of non-churned members who placed at least one bid on lots sold between Jan-March

In [192]:
data_not_churn['bid_atleast_once'] = np.where(data_not_churn['total_lots_placed_bids_on'] > 0, 1, 0)

In [168]:
data_not_churn[data_not_churn['bid_atleast_once']==1]['won_atleast_once'].value_counts()

won_atleast_once
1    1443
0     936
Name: count, dtype: int64

* List of non-churned members who won at least one lot between Jan-March

In [193]:
data_not_churn['won_atleast_once'] = np.where(pd.isna(data_not_churn['total_lots_won']), 0, 1)

In [167]:
data_not_churn['won_atleast_once'].value_counts()

won_atleast_once
1    1443
0     990
Name: count, dtype: int64

### Merging churned and non-churned members data

In [171]:
master_df = pd.concat([data_churn, data_not_churn], ignore_index=True)

In [172]:
master_df

,owner_mbr_nbr,total_bids_on_lots_sold,total_lots_placed_bids_on,total_lots_won,mbr_state,member_sgmnt_type,churn,bid_atleast_once,won_atleast_once
0,463585,1041,212,3,MN,Consumer,1,1,1
1,748790,102,19,<NA>,AR,Consumer,1,1,0
2,314004,34,4,<NA>,PA,Consumer,1,1,0
3,723585,0,0,<NA>,PA,Consumer,1,0,0
4,299919,0,0,<NA>,LA,Consumer,1,0,0
...,...,...,...,...,...,...,...,...,...
6412,305893,7313,1297,5,FL,Dealer,0,1,1
6413,241014,907,105,<NA>,AZ,Consumer,0,1,0
6414,573155,1997,257,1,CA,Dealer,0,1,1
6415,247892,532,108,<NA>,CA,General Business,0,1,0


In [174]:
master_df['churn'].value_counts()  # total unique members

churn
1    3984
0    2433
Name: count, dtype: int64

In [175]:
master_df.shape

(6417, 9)

In [178]:
master_df.isnull().sum()  # checking for null values

owner_mbr_nbr                   0
total_bids_on_lots_sold         0
total_lots_placed_bids_on       0
total_lots_won               4681
mbr_state                       0
member_sgmnt_type             342
churn                           0
bid_atleast_once                0
won_atleast_once                0
dtype: int64

In [179]:
master_df['total_lots_won'] = master_df['total_lots_won'].fillna(0)  # filling NaN values with 0

master_df['member_sgmnt_type'] = master_df['member_sgmnt_type'].fillna('NA')

In [180]:
master_df.isnull().sum()  # checking for null values

owner_mbr_nbr                0
total_bids_on_lots_sold      0
total_lots_placed_bids_on    0
total_lots_won               0
mbr_state                    0
member_sgmnt_type            0
churn                        0
bid_atleast_once             0
won_atleast_once             0
dtype: int64

In [182]:
master_df

,owner_mbr_nbr,total_bids_on_lots_sold,total_lots_placed_bids_on,total_lots_won,mbr_state,member_sgmnt_type,churn,bid_atleast_once,won_atleast_once
0,463585,1041,212,3,MN,Consumer,1,1,1
1,748790,102,19,0,AR,Consumer,1,1,0
2,314004,34,4,0,PA,Consumer,1,1,0
3,723585,0,0,0,PA,Consumer,1,0,0
4,299919,0,0,0,LA,Consumer,1,0,0
...,...,...,...,...,...,...,...,...,...
6412,305893,7313,1297,5,FL,Dealer,0,1,1
6413,241014,907,105,0,AZ,Consumer,0,1,0
6414,573155,1997,257,1,CA,Dealer,0,1,1
6415,247892,532,108,0,CA,General Business,0,1,0


In [187]:
master_df.to_csv('churned_members_data_new.csv', index=False)

In [186]:
master_df[master_df['owner_mbr_nbr'] == 959102] ## check one case

,owner_mbr_nbr,total_bids_on_lots_sold,total_lots_placed_bids_on,total_lots_won,mbr_state,member_sgmnt_type,churn,bid_atleast_once,won_atleast_once
5,959102,22374,3966,0,AR,Dealer,1,1,0


### Sample code to perform SHAP later

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
import shap

X = master_df.drop('churn', axis=1)
y = master_df['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)


In [ ]:
imbalance_ratio = (y_train == 0).sum() / (y_train == 1).sum()

# Train XGBoost with native categorical handling
model = XGBClassifier(tree_method='hist',
                      enable_categorical=True,
                      scale_pos_weight=imbalance_ratio,
                      use_label_encoder=False,
                      eval_metric='logloss',
                      random_state=42)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
##SHAP values

from sklearn.preprocessing import LabelEncoder

X_shap = X_test.copy()

for col in X_shap.select_dtypes(include='category').columns:
    le = LabelEncoder()
    X_shap[col] = le.fit_transform(X_shap[col])

explainer = shap.Explainer(model)
shap_values = explainer(X_shap)

shap.summary_plot(shap_values, X_test)

In [ ]:
model.feature_importances_